# Comparing GEP results
For each country, results are defined by the geographic and secnario summary files. In this script, results are downloaded, extracted, and analyzed to see how the scenarios affect results

In [1]:
import os, sys, importlib, shutil, zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import GEP
from GEP import gepResults

importlib.reload(GEP)

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
importlib.reload(GEP)
GEP.get_assumptions("0_0_0_0_1_0")

['Electricity demand target: Bottom-up',
 'Social & commercial uses: Productive included',
 'Grid generation cost: Estimated',
 'PV cost: Estimated',
 'Intermediate investment: Capped connections in 2025',
 'Rollout: Least-cost nationwide']

In [3]:
resultsFolder = "/media/gost/DATA1/GEP/Scenario_Summaries/"
shapesFolder = "/media/gost/DATA1/GEP/Clusters"
outFolder = "/media/gost/DATA1/GEP/Summaries_Summary"
country_summaries_folder = "/media/gost/DATA1/GEP/Country_Scenarios"
chart_folder = "/media/gost/DATA1/GEP/CHARTS/"

for x in [country_summaries_folder]:
    if not os.path.exists(country_summaries_folder):
        os.makedirs(country_summaries_folder)

In [4]:
# Create summary output table containing all the summary files stacked together
all_results = []
for root, dirs, files in os.walk(resultsFolder):
    for f in files:
        if "summary.csv" in f:
            all_results.append(os.path.join(root, f))

In [5]:
importlib.reload(GEP)
try:
    del(final)
except:
    pass
all_res = {}
broken_res = {}
for x in all_results:
    xx = GEP.gep_summary(x)
    tempD = xx.get_data()
    if tempD.shape[1] == 5:
        all_res[x.split("/")[-1][:16]] = tempD
        try:
            final = final.append(tempD)
        except:
            final = tempD
    else:
        broken_res[x.split("/")[-1][:16]] = tempD
        
final.columns = ['Attribute','2025','2030','Country','Scenario']
final['Attr'] = final['Attribute'].apply(lambda x: x[:1])
final = final.loc[final['Attr'] != 'M',]

In [6]:
importlib.reload(GEP)
final.reset_index(inplace=True)
final['Tech'] = final['Attribute'].apply(lambda x: GEP.get_simple_tech(x))
final['Tech_detail'] = final['Attribute'].apply(lambda x: GEP.get_tech(x))
final['2030Sum'] = final['2025'] + final['2030']
final['PP'] = final['Country'].apply(GEP.get_pp)
africa = final.loc[final['PP'] != "other"]

# Determine most expensive scenario per country

In [7]:
country_grouped = final.loc[final['Attr'] == '4'].groupby(['Country'])

In [8]:
all_res = []
for x in country_grouped:
    max_scenario = x[1].groupby('Scenario').sum().idxmax()['2030Sum']
    all_res.append([x[0]] + max_scenario.split("_") + [max_scenario])
max_scenarios = pd.DataFrame(all_res, columns = ["Country","Demand","Uses","GridCost","PVCost","Plan","Rollout",'Scenario'])    

all_res = []
for x in country_grouped:
    min_scenario = x[1].groupby('Scenario').sum().idxmin()['2030Sum']
    all_res.append([x[0]] + min_scenario.split("_") + [max_scenario])
min_scenarios = pd.DataFrame(all_res, columns = ["Country","Demand","Uses","GridCost","PVCost","Plan","Rollout",'Scenario'])    

In [9]:
max_scenarios['Demand'].value_counts()
max_scenarios['Uses'].value_counts()
max_scenarios['GridCost'].value_counts()
max_scenarios['PVCost'].value_counts()
max_scenarios['Plan'].value_counts()
max_scenarios['Rollout'].value_counts()

1    53
0     1
Name: Rollout, dtype: int64

# Calculate wide-averages
Summarize scenarios across Africa and across the entire datasets. We want to calculate the following

1. Total new connections between start and 2030 for all technologies  
   a. Africa only  
   b. Global  


In [10]:
res_folder = "/media/gost/DATA1/GEP/Analytic_Results"
if not os.path.exists(res_folder):
    os.makedirs(res_folder)

In [11]:
# Summarize all variables by technology across globe by default scenario
scenario = "0_0_0_0_1_0"
com = final.loc[final['Scenario'] == scenario].groupby(['Tech', 'Attr'])
glo_default_summary = com.sum()
glo_default_summary.to_csv(os.path.join(res_folder, f"GLO_{scenario}.csv"))

In [12]:
# Summarize all variables by technology across afr by default scenario
scenario = "0_0_0_0_1_0"
com = africa.loc[africa['Scenario'] == scenario].groupby(['Tech', 'Attr'])
afr_default_summary = com.sum()
afr_default_summary.to_csv(os.path.join(res_folder, f"AFR_{scenario}.csv"))

In [13]:
# Summarize all variables by least expensive scenario
filter_dict = min_scenarios.loc[:,['Country','Scenario']]
all_idx = []
for idx, row in filter_dict.iterrows():
    tempD = final.loc[final['Country'] == row['Country']]
    tempD = tempD.loc[tempD['Scenario'] == row['Scenario']]
    all_idx = all_idx + tempD.index.to_list()
    
min_data = final.loc[all_idx]

com = min_data.groupby(['Tech','Attr'])
glo_min_summary = com.sum()
glo_min_summary.to_csv(os.path.join(res_folder, f"GLO_MIN_Scenario.csv"))

min_data = min_data.loc[min_data['PP'] != "other"]
com = min_data.groupby(['Tech','Attr'])
afr_min_summary = com.sum()
afr_min_summary.to_csv(os.path.join(res_folder, f"AFR_MIN_Scenario.csv"))

In [14]:
# Summarize all variables by most expensive scenario
filter_dict = max_scenarios.loc[:,['Country','Scenario']]
all_idx = []
for idx, row in filter_dict.iterrows():
    tempD = final.loc[final['Country'] == row['Country']]
    tempD = tempD.loc[tempD['Scenario'] == row['Scenario']]
    all_idx = all_idx + tempD.index.to_list()
    
max_data = final.loc[all_idx]

com = max_data.groupby(['Tech','Attr'])
glo_max_summary = com.sum()
glo_max_summary.to_csv(os.path.join(res_folder, f"GLO_MAX_Scenario.csv"))

max_data = max_data.loc[max_data['PP'] != "other"]
com = max_data.groupby(['Tech','Attr'])
afr_max_summary = com.sum()
afr_max_summary.to_csv(os.path.join(res_folder, f"AFR_MAX_Scenario.csv"))

In [15]:
# Combine the scenario summaries
combined_results = glo_default_summary.copy()
combined_results.drop(['index','2025','2030'], axis=1, inplace=True)
combined_results.columns = ['GLO_Default']
combined_results['AFR_Default'] = afr_default_summary['2030Sum']
combined_results['GLO_Max'] = glo_max_summary['2030Sum']
combined_results['AFR_Max'] = afr_max_summary['2030Sum']
combined_results['GLO_Min'] = glo_min_summary['2030Sum']
combined_results['AFR_Min'] = afr_min_summary['2030Sum']

combined_results.to_csv(os.path.join(res_folder, "COMBO_Attribute_summaries.csv"))

In [ ]:
### Global summary of percentage of new connections that are MG
attr='2'
tech='SA'
### Summarize across technologies
ag = final.loc[(final['Attr'] == attr) & (final['Tech'] == tech)].groupby(['Scenario','Attr','Tech'])
summed_ag = ag.sum()
summed_ag = summed_ag.loc[summed_ag['2030Sum'] > 0]
max_mg_new_connections = summed_ag.sort_values(['2030Sum'], ascending=False).iloc[0,]
min_mg_new_connections = summed_ag.sort_values(['2030Sum'], ascending=True ).iloc[0,]
pd.DataFrame([max_mg_new_connections,min_mg_new_connections])

In [ ]:
### africa summary of percentage of new connections that are MG
attr='2'
tech='MG'
for attr in africa['Attr'].unique():
    for tech in africa['Tech'].unique():
        ### Summarize across technologies
        ag = africa.loc[(africa['Attr'] == attr) & (africa['Tech'] == tech)].groupby(['Scenario','Attr','Tech'])
        summed_ag = ag.sum()
        summed_ag = summed_ag.loc[summed_ag['2030Sum'] > 0]
        summed_ag.to_csv(os.path.join(res_folder, f"AFRICA_{attr}_{tech}.csv"))
'''
max_mg_new_connections = summed_ag.sort_values(['2030Sum'], ascending=False).iloc[0,]
min_mg_new_connections = summed_ag.sort_values(['2030Sum'], ascending=True ).iloc[0,]
pd.DataFrame([max_mg_new_connections,min_mg_new_connections])
'''

In [ ]:
### africa compare all technologies 
ag = africa.groupby(['Scenario','Attr','Tech'])
summed_ag = ag.sum()
summed_ag = summed_ag.loc[summed_ag['2030Sum'] > 0]
summed_ag.to_csv(os.path.join(res_folder, f"AFRICA_compare_everything.csv"))

In [ ]:
### global compare all technologies 
ag = final.groupby(['Scenario','Attr','Tech'])
summed_ag = ag.sum()
summed_ag = summed_ag.loc[summed_ag['2030Sum'] > 0]
summed_ag.to_csv(os.path.join(res_folder, f"GLOBAL_compare_everything.csv"))

In [ ]:
summed_ag

# Compare 2025 and 2030 new connections

In [ ]:
### Global summary of percentage of new connections that are MG
attr='2'
tech='MG'
### Summarize across technologies
ag = final.loc[(final['Attr'] == attr) & (final['Tech'] == tech)].groupby(['Country','Scenario','Attr','Tech'])
for name, group in ag:
    if group['2025'].sum() > group['2030'].sum() and name[1] == "1_1_0_1_1_0":
        break

# Create plots

In [ ]:
importlib.reload(GEP)
# Create plots describing all costs for all separate scenarios
attributes = joined_data.index.get_level_values(0)
scenarios = joined_data.index.get_level_values(1)
for selected_attribute in ['1','2','3','4']:
    for selected_scenario in scenarios:
        if selected_scenario[-1] == '0':
            GEP.extract_plot(joined_data, selected_attribute, selected_scenario, "Africa", chart_folder)
            GEP.extract_plot(wapp_data,   selected_attribute, selected_scenario, "WAPP", chart_folder)
            GEP.extract_plot(eapp_data,   selected_attribute, selected_scenario, "EAPP", chart_folder)
            GEP.extract_plot(sapp_data,   selected_attribute, selected_scenario, "SAPP", chart_folder)
            break

In [ ]:
importlib.reload(GEP)
# Create boxplots of scenarios for selected countries
countries = ['ht-1']
for country in countries:        
    inD = final.loc[final['Country'] == country]
    inD = inD.loc[inD['Scenario'].apply(lambda x: int(x[-1]) == 0)]
    chart_folder = '/media/gost/DATA1/GEP/CHARTS/Box_Plot'
    for selected_attribute in ['1','2','3','4']:
        res = GEP.box_plot(inD, selected_attribute, f'{chart_folder}/boxPlot_{country}_{selected_attribute}.png')
        

In [ ]:
wapp_data = pd.DataFrame(final[final['Country'].isin(WAPP)].groupby(['Attribute','Scenario']).sum())
wapp_data.reset_index(inplace=True)
eapp_data = pd.DataFrame(final[final['Country'].isin(EAPP)].groupby(['Attribute','Scenario']).sum())
eapp_data.reset_index(inplace=True)
sapp_data = pd.DataFrame(final[final['Country'].isin(SAPP)].groupby(['Attribute','Scenario']).sum())
sapp_data.reset_index(inplace=True)

for selected_attribute in ['1','2','3','4']:
    res = box_plot(wapp_data, selected_attribute, f'{chart_folder}/boxPlot_WAPP_{selected_attribute}.png')
    res = box_plot(eapp_data, selected_attribute, f'{chart_folder}/boxPlot_EAPP_{selected_attribute}.png')
    res = box_plot(sapp_data, selected_attribute, f'{chart_folder}/boxPlot_SAPP_{selected_attribute}.png')

# Summarize all scenarios for single country

In [ ]:
country = "mz"

In [ ]:
sel_data = final.loc[final['Country'] == country]
#remove scenarios from final lever
sel_grouped = sel_data.groupby(['Attr', 'Tech'])

In [ ]:
def get_vals(x):
    return(
        x['2025'].min(),
        x['2030'].max(),
        x['2025'].min(),
        x['2030'].max()
    )

    
#results = pd.DataFrame(sel_grouped.apply(get_vals))

results = pd.DataFrame([sel_grouped['2025'].min(),
                      sel_grouped['2025'].max(),
                      sel_grouped['2030'].min(),
                      sel_grouped['2030'].max(),
                     ]).transpose()
results.reset_index(inplace=True)
results.columns = ['Attr','Tech','2025min','2025max','2030min','2030max']
results

# Summarize scenarios across countries

In [ ]:
### Generate national summary table - as simple as possible
scenario = "1_1_1_1_1_0"

attr = '2'
summaryData = final.loc[(final['Scenario'] == scenario) & (final['Attr'] == attr),]
#summaryData.loc[summaryData['2025'] != summaryData['2030'],]

summaryData['Attr'] = summaryData['Attribute'].apply(lambda x: x[:1])
summaryData = summaryData.loc[summaryData['Attr'] != 'M',]
summaryData['Tech'] = summaryData['Attribute'].apply(lambda x: get_tech(x))
summaryData['PP'] = summaryData['Country'].apply(lambda x: get_pp(x))
summaryData['Continent'] = summaryData['Country'].apply(lambda x: get_continent(x)) 

#summary_grouped = summaryData.groupby(['Country','Attr','Tech'])
#pd.DataFrame(summary_grouped.mean()).to_csv(f"{chart_folder}_country_breakdown_{scenario}.csv")

summary_grouped = summaryData.groupby(['Continent','Attr','Tech'])
#pd.DataFrame(summary_grouped.mean()).to_csv(f"{chart_folder}_PP_breakdown_{scenario}.csv")
summary_grouped.sum()

In [ ]:
# Generate output table of Country with population (1) and investment (4) by grid and off grid
scenario = scenario #"0_0_0_0_0_0"
summaryData = final.loc[final['Scenario'] == scenario,]
summaryData['Attr'] = summaryData['Attribute'].apply(lambda x: x[:1])
summaryData = summaryData.loc[summaryData['Attr'] != 'M',]
summaryData['Tech'] = summaryData['Attribute'].apply(lambda x: get_tech(x))
summary_grouped = summaryData.groupby(['Country','Attr','Tech'])
pd.DataFrame(summary_grouped.mean()).to_csv(f"{chart_folder}_country_breakdown_GRID_OFFGRID.csv")


In [ ]:
# For a single country and scenario, generate description of scenario 
# with population (1) and investment (4) by grid, MG, off grid
scenario = "0_1_0_0_0_0"
country = 'bi-1'
out_file = os.path.join(country_summaries_folder, f"{country}_{scenario}")

summaryData = final.loc[(final['Scenario'] == scenario) & (final['Country'] == country),]
summary_grouped = summaryData.groupby(['Attr','Tech'])
grouped_vals = pd.DataFrame(summary_grouped.mean())

#Print outputs to screen and or file
grid_pop = grouped_vals.loc[(grouped_vals.index.get_level_values('Attr') == '1') & 
                            (grouped_vals.index.get_level_values('Tech') == 'GRID'),'2030'].iloc[0]
off_grid_pop = grouped_vals.loc[(grouped_vals.index.get_level_values('Attr') == '1') & 
                            (grouped_vals.index.get_level_values('Tech') == 'SA'),'2030'].iloc[0]
mini_grid_pop = grouped_vals.loc[(grouped_vals.index.get_level_values('Attr') == '1') & 
                            (grouped_vals.index.get_level_values('Tech') == 'MG'),'2030'].iloc[0]

grid_i = grouped_vals.loc[(grouped_vals.index.get_level_values('Attr') == '4') & 
                            (grouped_vals.index.get_level_values('Tech') == 'GRID'),'2030Sum'].iloc[0]
off_grid_i = grouped_vals.loc[(grouped_vals.index.get_level_values('Attr') == '4') & 
                            (grouped_vals.index.get_level_values('Tech') == 'SA'),'2030Sum'].iloc[0]
mini_grid_i = grouped_vals.loc[(grouped_vals.index.get_level_values('Attr') == '4') & 
                            (grouped_vals.index.get_level_values('Tech') == 'MG'),'2030Sum'].iloc[0]

scenario_description = GEP.get_assumptions(scenario)

messages = []
messages.append(f"***Scenario {scenario}***")
messages.append("\n".join(scenario_description))
messages.append("\n")
messages.append(f'2030 Grid pop: \t {round(grid_pop)} \t Grid Investment {round(grid_i)}')
messages.append(f'2030 Offgrid pop: \t {round(off_grid_pop)} \t Grid Investment {round(off_grid_i)}')
messages.append(f'2030 Mini grid pop: \t {round(mini_grid_pop)} \t Grid Investment {round(mini_grid_i)}')

if out_file != '':
    with open(out_file, 'w') as output_file:
        for m in messages:
            output_file.write(m)
            output_file.write("\n")

for m in messages:
    print(m)

In [ ]:
# Calculate the percentage of new connections that are GRID in each scenario
maxConnect = 0
minConnect = 1000
attr = '2' # Looking at number of grid connections
for scenario in final['Scenario'].unique():
    if scenario[-1] == '0':
        summaryData = final.loc[final['Scenario'] == scenario,]
        allConnect  = summaryData.loc[summaryData['Attr'] == attr,'2030Sum'].sum()
        gridConnect = summaryData.loc[summaryData['Attribute'] == '2.New_Connections_Grid', '2030Sum'].sum()
        gridTotal = gridConnect / allConnect
        if gridTotal > maxConnect:
            maxConnect = gridTotal
            maxScenario = scenario
        if gridTotal < minConnect:
            minConnect = gridTotal
            minScenario = scenario
        print(f'{scenario} - {gridTotal}')
print('***')
print(f'Maximum Grid: {maxScenario} - {maxConnect}')
print(f'Minimum Grid: {minScenario} - {minConnect}')

In [ ]:
# Calculate the percentage of new connections that are SA in each scenario in 2030
maxConnect = 0
minConnect = 1000
attr = '2' # Looking at number of grid connections
for scenario in final['Scenario'].unique():
    if scenario[-1] == '0':
        summaryData = final.loc[final['Scenario'] == scenario,]
        allConnect  = summaryData.loc[summaryData['Attr'] == attr,'2030'].sum()
        SAConnect = summaryData.loc[summaryData['Attribute'] == '2.New_Connections_SA_PV', '2030'].sum()
        SATotal = SAConnect / allConnect
        if SATotal > maxConnect:
            maxConnect = SATotal
            maxScenario = scenario
        if SATotal < minConnect:
            minConnect = SATotal
            minScenario = scenario
        print(f'{scenario} - {gridTotal}')
print('***')
print(f'Maximum SA: {maxScenario} - {maxConnect}')
print(f'Minimum SA: {minScenario} - {minConnect}')